# Root Finding Algorithms

* Bisection Method

* Newton-Raphson Method

* Secant Method

### Sub-routines (for multiple roots):

* Polynomial Deflation/Factoring - Only for Polynomials

### Other Methods:

* [Laguerre Method](https://en.m.wikipedia.org/wiki/Laguerre%27s_method) (A "sure-fire" method to successively calculate all the roots of polynomial functions)

## Bisection Method | Interval Halving Method | Binary Search Method (Similar, but unrelated to CS) | Dichotomy Method

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Bisection_method.svg/500px-Bisection_method.svg.png " width="400" title="Bisection Method">

#### Solving for 1 root of $f$ in $(a, b)$:

$$f(x) = 0, \qquad x \in \mathbb{R}$$

* $f$ is continuous over $[a, b]$ | $f(a)$ and $f(b)$ have opposite signs $\implies f$ has at least 1 root in $(a, b)$ (Using Intermediate Value Theorem).

#### **INPUTS**:

* Function, $f(x)$.

* Interval, $[a, b]$.

* Tolerance, $\epsilon$.

#### **STEPS**:

1. Calculate $c = \frac{a+b}{2}$ (Midpoint of the interval).

2. Calculate $f(c)$, the function value at the midpoint.

3. If convergence is satisfactory (that is, $|c - a|$ is sufficiently small ($< \epsilon$), or $|f(c)|$ is sufficiently small), return $c$ and stop iterating.

4. Examine the sign of $f(c)$ and replace either $(a, f(a))$ or $(b, f(b))$, with $(c, f(c))$, so that, there is a zero crossing, within the new interval.

### Notes:

* Convergence is Linear $\implies$ Costly to work with, for general functions | Usually, used to obtain an initial good guess and then, another (better) method is invoked to find the root(s).

* Assumes the presence of exactly 1 root, in the specified interval, and tries to find only 1 root.

* Applies to Continuous Functions

* Two values with opposing signs need to be specified

In [26]:
import numpy as np, scipy as sc, matplotlib.pyplot as plt

def rfa_bisect(f, a, b, tol=1e-6):
    """
    f: Function, whose root is to be found
    a: Lower bound of the Interval | SIDENOTE: "lower" and "upper" don't really matter
    b: Upper bound of the Interval
    tol: Tolerance value, for Convergence Check
    """
    c = (a+b)/2
    # Convergence Condition 
    if np.abs(f(c)) <= tol:
        return c
    elif f(c)*f(a) < 0: # Different Signs at c and a => Replace b
        return rfa_bisect(f, a, c, tol)
    elif f(c)*f(b) < 0: # Different Signs at c and b => Replace a
        return rfa_bisect(f, c, b, tol)
    elif np.sign(f(c)*f(a)) == np.sign(f(c)*f(b)):
        print("No root found, in the specified interval.")
        return None
             

# MODULE TEST - 0
f = lambda x: x**2 - 1 # Roots at |x| = 1, Only x = 1 in (-0.99999999, 1.000000001)
a = -0.99999999
b = 1.000000001

# MODULE TEST - 1
f1 = lambda x: x**4 - 4*x**3 - 2*x**2 + 12*x - 3 # Roots at x = \pm\sqrt{3}, 2\pm\sqrt{3}
a1 = 0
b1 = 2

print("Root of f, in the interval, ({}, {}): x = {}".format(a, b, rfa_bisect(f, a, b, tol=1e-8)))
print("Root of f1, in the interval, ({}, {}): x = {}".format(a1, b1, rfa_bisect(f1, a1, b1, tol=1e-8)))

Root of f, in the interval, (-0.99999999, 1.000000001): x = 0.9999999972747098
Root of f1, in the interval, (0, 2): x = 0.26794919185340405


## Newton-Raphson Method | Newton's Method

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e0/NewtonIteration_Ani.gif " width="400" title="Bisection Method">

$$\boxed{x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}}$$

### INPUTS:

* f: Function, whose root is to be found
* x0: Initial guess for Root
* iters: Number of iterations
* tol: Tolerance value, for Convergence Check
* step_for_deriv: Step-size for calculating derivatives | OPTIONAL Argument

### Convergence Issues:

* Convergence is highly sensitive to the Initial Guess (x0), as well as, the number of iterations (iters) and tolerance (tol).

* Beware of:
    * Poor Initial Estimates/Guesses
    * Using Stationary Points, as Initial Guesses
    * Starting point enters a cycle
    * Closely-spaced roots ('tol' plays a role here)
    * *Basins of Attraction*
    * Divergence, due to Overshoot (Could be due to accumulating approximation errors in derivative estimation). Especially, for the follwing class of functions:
    $$f(x) = |x|^a$$
        * For $0 \lt a \lt 0.5$, root is overshot and sequence diverges.
        * For $a = 0.5$, root is overshot, but sequence oscillates between two values.
        * For $0.5 \lt a \lt 1$, root is still overshot, but sequence converges.
        * For $a \ge 1$, root is not overshot and sequence converges.
    * Wiggly Functions
    * Discontinuity at or near Root

### Notes:

* Can be used to compute Square Roots, fairly easily.

* Order of Convergence, $\phi = 2$ (Quadratic, but not always (like in case of zero first derivative or no second derivative, at root) - highly situational).

* For roots, with multiplicity, $m \gt 1$, method of *successive over-relaxation* can be used.

* Calculating the first derivative is not always easy.

* Worse than Bisection Method and Regula Falsi, due to no bracketing of root in this method, implying no guarantee of convergence.

In [13]:
import numpy as np, scipy as sc, matplotlib.pyplot as plt

# Adapted from del_5p_f from ODEs.ipynb
def del_5p(x, func, step=1e-4):
    """
    Uses 5-point stencil to calculate the derivative at a point
    
    INPUT:
    x: Point, at which the function, is to be differentiated
    func: Function, to be differentiated
    step: Step-size | OPTIONAL
    OUTPUT:
    df: First Order Derivative at the specific point
    """
    df = (-func(x + 2*step) + 8*func(x + step) - 8*func(x - step) + func(x - 2*step))/(12*step)
    return df

def rfa_newton(f, x0, iters, tol=1e-6, step_for_deriv=1e-4):
    """
    f: Function, whose root is to be found
    x0: Initial guess for Root
    iters: Number of iterations
    tol: Tolerance value, for Convergence Check
    step_for_deriv: Step-size for calculating derivatives | OPTIONAL Argument
    """
    # Convergence Condition
    x = np.array([x0] + iters*[None], dtype=float)
    
    for i in range(iters):
        x[i+1] = x[i] - f(x[i])/(del_5p(x[i], f, step=1e-4))
        if np.abs(f(x[i+1])) <= tol:
            return x[i+1]
        
    print("Last calculated guess does not satisfy the Tolerance Limit, in the specified Number of Iterations.")
    return None
             

# MODULE TEST
f = lambda x: x**2 - x - 2 # Roots at |x| = 1, Only x = 1 in (-0.99999999, 1.000000001)
x0 = 10

# MODULE TEST
f1 = lambda x: x**4 - 4*x**3 - 2*x**2 + 12*x - 3 # Roots at x = \pm\sqrt{3}, 2\pm\sqrt{3}
x0_1 = -1e9
# a1 = -10
# b1 = 10

print("Root of f: x = {}".format(rfa_newton(f1, x0, iters=100, tol=1e-8)))

Root of f: x = 3.732050807578761


## Secant Method

### Recurrence Relation:
$$\boxed{x_{i+2} = x_{i+1} - f(x_{i+1})\frac{x_{i+1} - x_{i}}{f(x_{i+1}) - f(x_{i})} = \frac{x_{i}f(x_{i+1}) - x_{i+1}f(x_{i})}{f(x_{i+1}) - f(x_{i})}}$$

### INPUTS:

* f: Function, whose root(s) is/are to be found
* x0: First guess for the root
* x1: Second guess for the root
* iters: Number of Iterations
* tol: Tolerance Limit, for deviation from actual root

### Convergence Issues:

* Convergence is highly sensitive to the Initial Guesses (x0, x1), as well as, the number of iterations (iter) and tolerance (tol).

* Beware of:
    * Poor Initial Estimates/Guesses
    * Using Stationary Points, as Initial Guesses
    * Starting point enters a cycle
    * Closely-spaced roots ('tol' plays a role here)
    * *Basins of Attraction*
    * Divergence, due to Overshoot (Could be due to accumulating approximation errors in derivative estimation).
    * Wiggly Functions

### Notes:

* Quasi-Newton Method - Finite Difference Approximation to (the derivative in) Newton's Method (but predates Newton's Method by 3 Millenia).

* Order of Convergence, $\phi \approx 1.618$ (Golden Ratio) (Superlinear, but not Quadratic, like in Newton's Method).

* Situationally better than Newton's Method (whenever calculating actual derivatives is costlier), despite lower convergence rate.

* Worse than Bisection Method and Regula Falsi, due to no bracketing of root in this method, implying no guarantee of convergence.

In [2]:
import numpy as np, scipy as sc, matplotlib.pyplot as plt

def rfa_secant(f, x0, x1, iters, tol=1e-6):
    """
    f: Function, whose root is to be found
    x0: First guess for the root
    x1: Second guess for the root
    iters: Number of Iterations
    tol: Tolerance value, for Meshing & Convergence Check
    """
    # Array of successive "better" guesses for the root (ideally)
    x = np.array([x0, x1] + iters*[None], dtype=float)
    # Initial Guesses
    x[0], x[1] = x0, x1

    for i in range(iters):
        x[i+2] = x[i+1] - f(x[i+1])*((x[i+1] - x[i])/(f(x[i+1]) - f(x[i])))
        # Convergence Test
        if np.abs(f(x[i+2])) <= tol:
            return x[i+2]
    
    print("Last calculated guess does not satisfy the Tolerance Limit, in the specified Number of Iterations.")
    return None
    

# MODULE TEST
f1 = lambda x: x**2 - x - 2 # Roots at x = -1 and x = 2
x0 = 5
x1 = 10

print("Root of f: x = {}".format(x0, x1, rfa_secant(f1, x0, x1, iters=100, tol=1e-6)))

Root of f: x = 5


### SUB ROUTINE - Polynomial Deflation/Factorization

### INPUTS:

* Original Polynomial Function: $f(x)$
* 1 Root: $x = x_1$

### SCHEME:

* Deflation Step (Making use of the Fundamental Theorem of Algebra):

$$f(x) = (x - x_1)g(x) \implies \boxed{g(x) = \frac{f(x)}{(x - x_1)}}$$

* Now, find a root for $g(x)$.

* "Deflate" and Repeat.

### Notes:

* Only applicable to Polynomial Functions.

* Can be used in conjunction with methods, that find single roots, to find other roots.

* Forms a step in [Laguerre Method](https://en.m.wikipedia.org/wiki/Laguerre%27s_method)

In [22]:
def deflate(f, x1):
    """
    INPUT:
    f: Function, to be deflated
    x1: Root of the function
    OUTPUT:
    Lambda function: Deflated f
    """
    return lambda x: f(x)/(x - x1)

#### MODULE TEST FOR DEFLATION

In [28]:
import numpy as np, scipy as sc, matplotlib.pyplot as plt

# Adapted from del_5p_f from ODEs.ipynb
def del_5p(x, func, step=1e-4):
    """
    Uses 5-point stencil to calculate the derivative at a point
    
    INPUT:
    x: Point, at which the function, is to be differentiated
    func: Function, to be differentiated
    step: Step-size | OPTIONAL
    OUTPUT:
    df: First Order Derivative at the specific point
    """
    df = (-func(x + 2*step) + 8*func(x + step) - 8*func(x - step) + func(x - 2*step))/(12*step)
    return df

def rfa_newton(f, x0, iters, tol=1e-6, step_for_deriv=1e-4):
    """
    f: Function, whose root is to be found
    x0: Initial guess for Root
    iters: Number of iterations
    tol: Tolerance value, for Convergence Check
    step_for_deriv: Step-size for calculating derivatives | OPTIONAL Argument
    """
    # Convergence Condition
    x = np.array([x0] + iters*[None], dtype=float)
    
    for i in range(iters):
        x[i+1] = x[i] - f(x[i])/(del_5p(x[i], f, step=1e-4))
        if np.abs(f(x[i+1])) <= tol:
            return x[i+1]
        
    print("Last calculated guess does not satisfy the Tolerance Limit, in the specified Number of Iterations.")
    return None
    
def find_all_roots(f, x0, num_roots, iters, tol=1e-6):
    """
    f: Polynomial, whose roots are to be found
    x0: Initial Guess
    
    num_roots: Number of roots in (a, b)
    tol: Tolerance value, for Convergence Check (in the single-root-finding function)
    """
    root = rfa_newton(f, x0, iters=100) # First Discovered Root
    print("Root = {}".format(root)) # Printing out discovered roots
    
    g = deflate(f, root)
    
    if num_roots > 1: # > 1, as 1 root has already been discovered
        find_all_roots(g, x0, num_roots-1, tol)
        
# MODULE TEST
f = lambda x: x**4 - 4*x**3 - 2*x**2 + 12*x - 3 # Roots at x = \pm\sqrt{3}, 2\pm\sqrt{3}
x0 = 10

find_all_roots(f, x0, num_roots=4, iters=100, tol=1e-8)

Root = 3.732050807578761
Root = 1.7320508075711785
Root = 0.2679491925126542
Root = -1.7320508075947032
